In [72]:
import sys
import os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = load_mnist( normalize=True, one_hot_label=True)
print (x_train.shape)
print (t_train.shape)
t_train

(60000, 784)
(60000, 10)


array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

In [73]:
train_size = x_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size)
print (batch_mask)
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]

[29699 26150  4099 56826 45495 25716 38029 34932 18917 43392]


In [74]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)

    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

In [75]:
y = np.random.rand(5, 10)
print(y.shape)
print(y)
t = [2, 7, 0, 9, 4]
y[np.arange(5), t]

(5, 10)
[[0.22407177 0.17812333 0.40175007 0.94907054 0.26758223 0.69046162
  0.48214355 0.39812087 0.37323117 0.9146793 ]
 [0.05075365 0.15971662 0.485994   0.95938191 0.99734973 0.85753522
  0.4966319  0.29966131 0.71546147 0.90694508]
 [0.092133   0.00837086 0.82117578 0.04681661 0.94306786 0.13433742
  0.19111267 0.12929854 0.87909881 0.86914923]
 [0.57843903 0.39986811 0.13063888 0.98294615 0.09825654 0.7342689
  0.84272448 0.72508874 0.19366037 0.76444961]
 [0.83520101 0.74990863 0.38714285 0.23325149 0.2873743  0.64556894
  0.31831407 0.35024624 0.73735291 0.88640523]]


array([0.40175007, 0.29966131, 0.092133  , 0.76444961, 0.2873743 ])

In [76]:
import sys
import os
sys.path.append(os.pardir)
import numpy as np
from common.functions import softmax, cross_entropy_error
from common.gradient import numerical_gradient

class simpleNet:
    def __init__(self):
        self.W = np.random.randn(2, 3)

    def predict(self, x):
        return np.dot(x, self.W)
    
    def loss(self, x, t):
        z = self.predict(x)
        y = softmax(z)
        loss = cross_entropy_error(y, t)

        return loss

x = np.array([0.6, 0.9])
t = np.array([0, 0, 1])

net = simpleNet()

In [77]:
net = simpleNet()
print(net.W)

[[ 0.3108283   1.38353201  0.0877706 ]
 [-0.02649785 -0.96823855 -0.32303145]]


In [78]:
x = np.array([0.6, 0.9])
p = net.predict(x)
print(p)


[ 0.16264892 -0.04129549 -0.23806594]


In [79]:
np.argmax(p)

np.int64(0)

In [80]:
t = np.array([0, 0, 1])
net.loss(x, t)


np.float64(1.3111274768713144)

In [81]:
x = np.array([0.6, 0.9])
t = np.array([0, 0, 1])
def f(w):
    return net.loss(x, t)
dW = numerical_gradient(f, net.W)

print(dW)

[[ 0.24141472  0.19687557 -0.43829029]
 [ 0.36212208  0.29531335 -0.65743544]]


In [1]:
import sys
import os
sys.path.append(os.pardir)
import numpy as np
from common.functions import sigmoid, sigmoid_grad, softmax
from common.gradient import numerical_gradient

class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init=0.01):
        self.params = {}
        self.params['W1'] = weight_init * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']

        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)

        return y
    
    def loss(self, x, t):
        y = self.predict(x)
        return cross_entropy_error(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)

        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
    

    def numerical_gradient(self, x, t):
        def loss_W(W):
            return self.loss(x, t)

        grads = {}

        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        return grads
    
    def gradient(self, x, t):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        grads = {}
        
        batch_num = x.shape[0]
        
        # forward
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        # backward
        dy = (y - t) / batch_num
        grads['W2'] = np.dot(z1.T, dy)
        grads['b2'] = np.sum(dy, axis=0)
        
        dz1 = np.dot(dy, W2.T)
        da1 = sigmoid_grad(a1) * dz1
        grads['W1'] = np.dot(x.T, da1)
        grads['b1'] = np.sum(da1, axis=0)

        return grads

In [2]:
import sys
import os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist
import matplotlib.pyplot as plt

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

train_loss_list = []

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    # grad = network.numerical_gradient(x_batch, t_batch)
    grad = network.gradient(x_batch, t_batch)
    
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]

    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train acc, test acc | " + str(train_acc) + ", " + str(test_acc))

markers = {'train': 'o', 'test': 's'}
x = np.arange(len(train_acc_list))
plt.plot(x, train_acc_list, label='train acc')
plt.plot(x, test_acc_list, label='test acc', linestyle='--')
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.ylim(0, 1.0)
plt.legend(loc='lower right')
plt.show()

NameError: name 'softmax' is not defined